In [1]:
!pip install -q langchain
!pip install -q llama_index
!pip install -q llama-index-llms-llama-cpp
!pip install -q chromadb
!pip install -q sentence-transformers

In [2]:
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader , CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import os

In [3]:
from langchain_community.llms import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="..\models\mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    verbose=True,  # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ..\models\mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:        

In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

#embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


# load the embedding model locally
from sentence_transformers import SentenceTransformer
embed_model_Path = "..\models\sentence-transformers/all-mpnet-base-v2"

#embed_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
#embed_model.save(modelPath)
#embed_model = SentenceTransformer(embed_model_Path)

embed_model = HuggingFaceEmbeddings(model_name=embed_model_Path)

c:\Users\lemon\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## Importing Libraries

from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [35]:
#
documents = SimpleDirectoryReader("../Data/txt").load_data()
documents

[Document(id_='6855057d-1fc6-4ac6-836f-f8d858f0c83e', embedding=None, metadata={'file_path': 'c:\\Users\\lemon\\OneDrive\\Documents\\GitHub\\Mistral-7b-Fine-tuning---RAG\\Data\\txt\\mini_data.csv', 'file_name': 'mini_data.csv', 'file_type': 'application/vnd.ms-excel', 'file_size': 3880, 'creation_date': '2024-05-22', 'last_modified_date': '2024-05-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='The credit card with the id of 2 is a Distinguish with the number of 55552127249722 and the expiration date of 8/2005 and it was last modified on 2013-12-05... \nThe credit card with the id of 3 is a ColonialVoice with the number of 77778344838353 and the expiration date of 7/2005 and it was last modified on 2014-01-14.\nThe credit card with the id of 4 is a

In [8]:
system_prompt = """
you are a Q&A assistant. your job is to answer questions accurately using the information provided to you.
"""

## default format supported by llama2
query_wrapper = SimpleInputPrompt("<USER>{query_str}<ASSISTANT>")

In [10]:
!pip install -q llama-index-llms-langchain

In [16]:
service_context=ServiceContext.from_defaults(
    chunk_size=160,
    chunk_overlap=40,
    llm=llm,
    embed_model=embed_model
)

C:\Users\lemon\AppData\Local\Temp\ipykernel_18152\1258871742.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context=ServiceContext.from_defaults(


In [27]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context,show_progress=True)

Generating embeddings: 100%|██████████| 13/13 [00:01<00:00,  6.52it/s]


In [28]:
query_engine = index.as_query_engine()
query_engine

[NodeWithScore(node=TextNode(id_='60550072-2b80-4af9-863c-2b54e4543c63', embedding=None, metadata={'file_path': 'c:\\Users\\lemon\\OneDrive\\Documents\\GitHub\\Mistral-7b-Fine-tuning---RAG\\Data\\txt\\mini_data.csv', 'file_name': 'mini_data.csv', 'file_type': 'application/vnd.ms-excel', 'file_size': 3880, 'creation_date': '2024-05-22', 'last_modified_date': '2024-05-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cf5ba895-3bcb-4e96-8ae3-241148b999e9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'c:\\Users\\lemon\\OneDrive\\Documents\\GitHub\\Mistral-7b-Fine-tuning---RAG\\Data\\txt\\mini_data.csv', 'file_name': 'mini_data.csv', 'file_type': 'application/vnd.ms-excel', 'file_size': 3880

In [29]:
response = query_engine.query("What are the credit card informations of the card with the id of 22?").get_formatted_sources()

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1999.65 ms
llama_print_timings:      sample time =      13.34 ms /    68 runs   (    0.20 ms per token,  5098.60 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (-nan(ind) ms per token, -nan(ind) tokens per second)
llama_print_timings:        eval time =   11761.24 ms /    68 runs   (  172.96 ms per token,     5.78 tokens per second)
llama_print_timings:       total time =   11948.71 ms /    68 tokens


In [30]:
print(response)

> Source (Doc id: 60550072-2b80-4af9-863c-2b54e4543c63): The credit card with the id of 10 is a SuperiorCard with the number of 33332126386493 and the exp...

> Source (Doc id: d9b5ef45-8115-473c-8921-5eff5abb68d5): The credit card with the id of 24 is a Vista with the number of 11111658051128 and the expiration...
